# Firework Algorithm for Optimization

- Autores originais: [*Ying Tan*](https://scholar.google.com/citations?user=PjNxSPsAAAAJ&hl=en) e [*Yuanchun Zhu*](https://ieeexplore.ieee.org/author/38008573100)
- Ano de Publicação: 2010
- Editora: [Springer](https://link.springer.com/chapter/10.1007/978-3-642-13495-1_44)
- Conferência: [Advances in Swarm Intelligence](https://link.springer.com/book/10.1007/978-3-642-13495-1): First International Conference, ICSI 2010, Beijing, China, June 12-15, 2010, Proceedings, Part I 1

## Descrição:
FA é um algoritmo de inteligência de enxames inspirado na explosão de fogos de artifício. Esse algoritmo é utilizado para a otimização global de funções complexas. São fornecidos dois processos de *“explosão”* (busca) e os mecanismos para manter a diversidade das *“faíscas”*.

In [1]:
import numpy as np
import pandas as pd
import random

## Funções Objetivos

In [2]:
def sphere_function(x):
    result = x**2
    return np.sum(result)

In [3]:
def rastrigin(x):
    n = len(x)
    total = 0
    total = x**2 - 10*np.cos(2*np.pi*x)
    return (10*n) + total

In [4]:
def rosenbrook(x):
    n = len(x)
    term1 = 100 * (x[1:] - x[:-1]**2)**2
    temr2 = (x[1:] -1)**2
    sumterm = np.sum(term1 + term2)
    return sumterm

In [5]:
def ackley(x):
    n = len(x)
    sum1 = np.sum(x*2)
    sum2 = np.sum(np.cos(2 * np.pi * x**2))
    term1 = -20 * np.exp(-0.2 * np.sqrt(1/n * sum1))
    term2 = -np.exp(1/n * sum2)
    return term1 + term2 + 20 + np.exp(1)

### Distribuição Gaussiana:

In [6]:
value = random.gauss(1,1)
value

-0.633132664431415

### Distribuição Uniforme:

In [7]:
value = random.uniform(-1,1)
value

0.7012413762962668

## Inicializando posições dos Fogos de Artifício:

In [33]:
def initialize_firework_locations(n, d, xmin, xmax):
    fireworks = xmin + (xmax - xmin) * np.random.rand(n,d)
    return fireworks

In [9]:
fireworks = initialize_firework_locations(5, -5.12, 5.12)
fireworks

array([ 1.17625644, -3.04062664,  2.18118833,  3.42343554,  5.00569121])

## Calculando o número de faíscas:

In [28]:
def sparks_number(fx, fX, m, ymax, eps=.00001):
    sum_difference_of_worst_fit_and_fx = np.sum(ymax-fX) + eps
    difference_of_max_and_fx = (ymax - fx + eps)
    s = m * difference_of_max_and_fx/sum_difference_of_worst_fit_and_fx
    s = firework_boundaries(0.1, 0.8, m, s)
    return s

In [11]:
def firework_boundaries(a, b, m, s):
    term1 = np.round(a*m) * (s < (a*m))
    term2 = np.round(b*m) * (s> b*m)
    term3 = np.round(s) * (~(s < (a*m)) & ~((s > b *m) & (a<b<1)))
    s_new = term1 + term2 + term3
    return s_new

## Obtendo localização das Faíscas:

### Seletor de dimensões aleatório

In [12]:
def random_dimension_selector(d):
    xj = np.zeros(d)
    z = round(d * random.uniform(0,1))
    random_dimensions = np.random.choice(d, z, replace=False)
    for i in random_dimensions:
        xj[i] = 1
    return xj
    

### Amplitude de explosão

In [13]:
def explosion_amplitude(A_hat, fx,fX, ymin, eps=.00001):
    sum_difference_of_fx_and_best_fit = np.sum(fX - ymin) + eps
    difference_of_fx_and_min = (fx - ymin + eps)
    A = A_hat * difference_of_fx_and_min/sum_difference_of_fx_and_best_fit
    return A

In [14]:
d = 6
round(6 * random.uniform(0,1))

4

### Obtendo faísca: Estratégia da Explosão

In [15]:
def obtain_location_of_spark(d, amp, xmin, xmax, xi):
    #selecting z dimensions to be affected
    xj = xi
    random_selected_positions = random_dimension_selector(d) 
    h = amp * random.uniform(-1 ,1) 
    random_affected_dimensions = xj + (random_selected_positions* h)
    #mapping affected dimensions to potential space
    random_affected_dimensions = map_to_potential_space(xmin, xmax, random_affected_dimensions)
    return random_affected_dimensions

### Mapeamento para espaço potencial:

In [16]:
def map_to_potential_space(xmin, xmax, affected):
    inbound = xmin + abs(affected)%(xmax - xmin)
    term1 = (affected > xmax) * inbound
    term2 =  (affected < xmin) * inbound
    fixed_bounds = term1 + term2
    mapping = (fixed_bounds == 0) * affected + fixed_bounds 
    return mapping

In [17]:
xi = np.random.rand(3)
xmin = np.array([-3.12, -4.5, -6.6])
xmax = np.array([3.12, 4.5, 6.6])
d = 3
amp = 5
obtain_location_of_spark(d, amp, xmin, xmax, xi)

array([ 0.47500779, -2.80197484,  0.13038762])

### Obtendo faísca: Estratégia de Faíscas Gaussianas:

In [18]:
def gaussian_spark_location(xmin, xmax, d, xi):
    xj = xi
    random_dimensions_selected = random_dimension_selector(d)
    random_affected_dimensions = (xj * np.logical_not(random_dimensions_selected)) + xj * (random_dimensions_selected * np.random.normal(1,1,d))
    random_affected_dimensions = map_to_potential_space(xmin, xmax, random_affected_dimensions)
    return random_affected_dimensions

In [19]:
d =3
np.random.normal(1,1,d)

array([1.16001331, 0.94374046, 2.03799127])

In [20]:
np.random.rand(3)

array([0.99937888, 0.53732951, 0.9730301 ])

## Seleção de Localizações:

### Calculando Distância Geral:

In [21]:
def general_distance(xi , X, d):
    sum_of_distances = np.sum(abs(xi - X))
    return sum_of_distances

### Calculando probabilidade de seleção:

In [22]:
def location_probability(RX, Rxi,d):
    pxi = Rxi/RX
    return pxi

## Framework ALgoritmo de Otimização Fogos de Artifício:

In [105]:
#localizações
n = 10
#quantidade de faíscas
m = 50
#dimensões
d = 10
#parâmetro para amplitude
a = 20
#faíscas gaussianas
gauss = 15

#Número de fogos na estratégia gaussiana
m_hat = 5

#minimo valor do espaço potencial
xmin = -5

#máximo valor do espaço potencial
xmax = 5

#Inicializando fogoso de artifício
X = initialize_firework_locations(n, d, xmin, xmax)

#Total de iterações
explosion_generations = 1000

#f(X)
y = np.zeros((n,1))
for i in range(n):
    y[i] = sphere_function(X[i])

#best fit
ymin = y.min()
#worst fit
ymax = y.max()

#Numero de faíscas por fogo de artifício
S = np.zeros((n,1))
#Amplitude de cada fogo de artificio
A = np.zeros((n,1))

for i in range(n):
    S[i] = sparks_number(y[i],y, m, ymax) 
    A[i] = explosion_amplitude(a, y[i],y, ymin)

total_sparks = np.sum(S)
new_locations = X
generation = np.array((n,d))
for i in range(n):
    xi = X[i]
    n_sparks = int(S[i][0])
    sparks = np.zeros((n_sparks,d))
    for j in range(n_sparks):
        sparks[j] = obtain_location_of_spark(d, a, xmin, xmax, xi)
    new_locations = np.vstack((new_locations, sparks)) 

random_fireworks = np.sort(np.random.choice(n,m_hat, replace=False))
for i in range(m_hat):
    xi = X[random_fireworks[i]]
    gaussian_sparks = np.zeros((gauss,d))
    for j in range(gauss):
        gaussian_sparks[j] = gaussian_spark_location(xmin, xmax, d, xi)
    new_locations = np.vstack((new_locations, gaussian_sparks)) 
total_locations = int(total_sparks) + n + m_hat*gauss
fx_new_locations = np.zeros((total_locations, 1))
for i in range(total_locations):
    fx_new_locations[i] = sphere_function(new_locations[i])

current_best_location = fx_new_locations.argmin()
best_location = new_locations[current_best_location]
new_locations = np.delete(new_locations, current_best_location,0)
RX = np.zeros((total_locations -1, 1)) 
for i in range(total_locations-1):
    RX[i] = general_distance(new_locations[i], new_locations, d)

prob = np.zeros((total_locations, 1))
print(total_locations)
print(new_locations.shape)
print(RX.shape)
print(X[0])
sum_RX = np.sum(RX)
for i in range(total_locations-1):
    prob[i] = location_probability(sum_RX, RX[i],d)
print((prob.shape))
# np.zeros((int(total_sparks) + n + m_hat*gauss, d))
# fx_new_locations = sphere_function(new_locations)
# fx = y[0]
# print("example X[0]: ", X[0])
# print("objective function f(X[0]): ", y[0])
# print("sparks per firework:\n", S)
# print("amplitude of fireworks: \n", A)
# print("Quantity of sparks\n",total_fireworks)  
# print("Quantity of gaussian sparks:\n ", gauss)
# print("Amplitude of firework\n",A.shape)
# print("new locations shape \n", new_locations.shape)
# print("total locations: ",total_sparks + n + m_hat*gauss)
# print("fx_best", fx_new_locations)

147
(146, 10)
(146, 1)
[ 1.66067785 -0.34623422  1.8755539  -2.9147944   3.04501523 -4.12481721
  4.26525196 -2.20725453  1.4993986   3.23262845]
(147, 1)


In [54]:
fx = y[4]
si = sparks_number(fx, y, m, ymax)
print("si: ",si)
print("fx: ", fx)
print("ymax: ",ymax)
sum = np.sum(ymax - y)
print("sum: ",sum)
spark =  (ymax - fx)/sum
print("ymax - y[0]/sum: ", spark)
print("m * ymax - fx/sum: ",m* spark)

si:  [5.]
fx:  [104.36463033]
ymax:  182.87446905467897
sum:  874.6521453791307
ymax - y[0]/sum:  [0.08976121]
m * ymax - fx/sum:  [4.48806072]


In [25]:
total_fireworks = np.sum(S)
total_fireworks

68.0

In [26]:
np.sort(np.random.choice(10,5, replace=False))

array([0, 2, 3, 6, 8])

In [27]:
xi = X[0]
gaussian_spark_location(xmin, xmax, d, xi)

array([-2.95871247, -4.93575806,  0.4718096 ,  1.34814434, -3.17250486,
       -3.38159368, -0.80918401,  0.18070943, -0.90108277, -1.76472518])

In [52]:
new_locations[101]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [69]:
new_locations[69:]

array([[-4.85284209e+00,  1.32871135e+00, -4.31490190e+00,
        -2.46195591e+00,  1.60816691e+00,  4.38218834e+00,
         4.43305852e+00, -3.64379314e+00,  4.60038077e+00,
         4.02160485e+00],
       [ 1.34558145e-01,  1.32871135e+00, -4.31490190e+00,
        -2.46195591e+00, -4.68214656e+00, -6.30411428e-01,
         4.43305852e+00, -3.64379314e+00, -4.12218999e-01,
         4.02160485e+00],
       [-4.85284209e+00,  1.32871135e+00, -4.31490190e+00,
        -2.46195591e+00,  3.30453206e-01,  4.38218834e+00,
         4.43305852e+00, -3.64379314e+00,  4.60038077e+00,
         4.02160485e+00],
       [-4.85284209e+00,  1.32871135e+00, -4.31490190e+00,
        -2.46195591e+00, -2.68682653e+00,  4.38218834e+00,
         4.43305852e+00, -3.64379314e+00,  4.60038077e+00,
         4.02160485e+00],
       [-4.85284209e+00,  1.32871135e+00, -4.31490190e+00,
        -2.46195591e+00,  3.30453206e-01,  4.38218834e+00,
         4.43305852e+00, -3.84717257e+00,  4.39700133e+00,
         4.

In [88]:
current_best_location

array([ 1.0718881 , -0.39035607, -0.37146852, -0.39237122,  1.1058866 ,
        0.74441681, -0.72170397,  0.81595826, -2.04869278,  2.0996152 ])